In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Configuración de estilo
sns.set_style("whitegrid")

## 1. Carga de Datos

In [ ]:
# Cargar datos
df = pd.read_excel("resultados.xlsx")

# Eliminar fila de "Total" si existe
df = df[df["Unidad"] != "Total"].copy()

# Verificar que la columna S_herb existe
if 'S_herb' not in df.columns:
    print("¡ADVERTENCIA! La columna 'S_herb' no se encuentra en el archivo.")
    print("Columnas disponibles:", df.columns.tolist())
else:
    print("Datos cargados correctamente. Primeras filas:")
    print(df[['Unidad', 'S_herb', 'Prc_expuesto']].head())

## 2. Análisis Exploratorio (Teoría: Detección de Sobredispersión)

Para decidir entre Poisson y Binomial Negativa, debemos verificar la relación entre la media y la varianza.
- **Poisson**: Asume que $Media \approx Varianza$ (Equidispersión).
- **Binomial Negativa**: Asume que $Varianza > Media$ (Sobredispersión).

Si la varianza es mucho mayor que la media, el modelo de Poisson subestimará los errores estándar, llevando a conclusiones erróneas (valores p significativos que no lo son).

In [ ]:
variable_respuesta = 'S_herb'

# 1. Histograma
plt.figure(figsize=(10, 6))
sns.histplot(df[variable_respuesta], kde=False, bins=15, color='lightgreen')
plt.title(f'Distribución de {variable_respuesta}')
plt.xlabel(variable_respuesta)
plt.ylabel('Frecuencia')
plt.show()

# 2. Cálculo de Media y Varianza
media = df[variable_respuesta].mean()
varianza = df[variable_respuesta].var()
ratio = varianza / media

print(f"Estadísticas para {variable_respuesta}:")
print(f"Media: {media:.4f}")
print(f"Varianza: {varianza:.4f}")
print(f"Ratio Varianza/Media: {ratio:.4f}")

if ratio > 1.5:
    print("\nINTERPRETACIÓN: La varianza es mayor que la media (Ratio > 1.5).")
    print("Esto sugiere SOBREDISPERSIÓN. Probablemente la Binomial Negativa sea mejor.")
elif ratio < 0.9:
    print("\nINTERPRETACIÓN: La varianza es menor que la media (Subdispersión).")
else:
    print("\nINTERPRETACIÓN: La varianza es similar a la media.")
    print("El modelo de Poisson podría ser adecuado.")

## 3. Ajuste de Modelos (GLM)

Ajustaremos ambos modelos usando `Prc_expuesto` (Apertura) como variable predictora.

In [ ]:
# Definir la fórmula
formula = f"{variable_respuesta} ~ Prc_expuesto"

# --- Modelo Poisson ---
modelo_poisson = smf.glm(formula=formula, data=df, family=sm.families.Poisson()).fit()

# --- Modelo Binomial Negativo ---
# Statsmodels requiere un parámetro alpha para la Binomial Negativa en GLM, 
# pero smf.negativebinomial estima la dispersión automáticamente.
modelo_nb = smf.negativebinomial(formula=formula, data=df).fit()

print("--- Resumen Modelo Poisson ---")
print(modelo_poisson.summary())
print("\n" + "="*80 + "\n")
print("--- Resumen Modelo Binomial Negativo ---")
print(modelo_nb.summary())

## 4. Comparación de Modelos

Usamos AIC (Criterio de Información de Akaike) y BIC. **Menor valor es mejor.**

In [ ]:
resultados = pd.DataFrame({
    'Modelo': ['Poisson', 'Binomial Negativa'],
    'AIC': [modelo_poisson.aic, modelo_nb.aic],
    'BIC': [modelo_poisson.bic, modelo_nb.bic],
    'Log-Likelihood': [modelo_poisson.llf, modelo_nb.llf]
})

print(resultados)

mejor_modelo = resultados.loc[resultados['AIC'].idxmin(), 'Modelo']
print(f"\nBasado en el AIC, el mejor modelo es: {mejor_modelo}")

## 5. Visualización del Ajuste

Graficamos los valores predichos por ambos modelos sobre los datos reales.

In [ ]:
# Predicciones
df['pred_poisson'] = modelo_poisson.predict(df)
df['pred_nb'] = modelo_nb.predict(df)

# Ordenar para graficar líneas suaves
df_sorted = df.sort_values('Prc_expuesto')

plt.figure(figsize=(10, 6))
# Datos reales
sns.scatterplot(x='Prc_expuesto', y=variable_respuesta, data=df, color='black', label='Datos Reales', alpha=0.6)

# Predicciones Poisson
plt.plot(df_sorted['Prc_expuesto'], df_sorted['pred_poisson'], 'r--', label='Ajuste Poisson', linewidth=2)

# Predicciones Binomial Negativa
plt.plot(df_sorted['Prc_expuesto'], df_sorted['pred_nb'], 'b-', label='Ajuste Binomial Negativa', linewidth=2)

plt.title(f'Ajuste de Modelos para {variable_respuesta}')
plt.xlabel('Porcentaje Expuesto (Apertura)')
plt.ylabel(variable_respuesta)
plt.legend()
plt.show()